In [8]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re

tweets_df = pd.read_csv("realdonaldtrump.csv")
def combine_text(row):
    content = row['content'] if pd.notna(row['content']) else ''
    hashtags = row['hashtags'] if pd.notna(row['hashtags']) else ''
    mentions = row['mentions'] if pd.notna(row['mentions']) else ''
    return f"{content} {hashtags} {mentions}"

tweets_df["text_full"] = tweets_df.apply(combine_text, axis=1)

def clean_tweet_for_vader(text):
    text = re.sub(r"http\S+|www.\S+", "", text)     
    text = re.sub(r"#", "", text)                   
    text = re.sub(r"@", "", text)                   
    text = re.sub(r"\s+", " ", text).strip()        
    return text


tweets_df["text_clean"] = tweets_df["text_full"].apply(clean_tweet_for_vader)

analyzer = SentimentIntensityAnalyzer()
tweets_df["vader_scores"] = tweets_df["text_clean"].apply(analyzer.polarity_scores)
tweets_df = pd.concat([tweets_df, tweets_df["vader_scores"].apply(pd.Series)], axis=1)

def classify_sentiment(score):
    if score >= 0.05:
        return "positive"
    elif score <= -0.05:
        return "negative"
    else:
        return "neutral"

tweets_df["sentiment_label"] = tweets_df["compound"].apply(classify_sentiment)

def sentiment_to_number(sentiment):
    if sentiment == 'neutral':
        return 0
    elif sentiment == 'positive':
        return 1
    else:
        return 2

In [11]:

trainable_data_df = tweets_df[['text_clean', 'date', 'retweets', 'favorites', 'sentiment_label']].copy()
trainable_data_df['sentiment_label'] = trainable_data_df['sentiment_label'].apply(sentiment_to_number)

trainable_data_df

,text_clean,date,retweets,favorites,sentiment_label
0,Be sure to tune in and watch Donald Trump on L...,2009-05-04 13:54:25,510,917,1
1,Donald Trump will be appearing on The View tom...,2009-05-04 20:00:10,34,267,1
2,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 08:38:08,13,19,1
3,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 15:40:15,11,26,0
4,"""My persona will never be that of a wallflower...",2009-05-12 09:07:28,1375,1945,0
...,...,...,...,...,...
43347,Joe Biden was a TOTAL FAILURE in Government. H...,2020-06-17 19:00:32,23402,116377,-1
43348,Will be interviewed on seanhannity tonight at ...,2020-06-17 19:11:47,11810,56659,1
43349,pic.twitter.com/3lm1spbU8X,2020-06-17 21:27:33,4959,19344,0
43350,pic.twitter.com/vpCE5MadUz,2020-06-17 21:28:38,4627,17022,0


Amazing product! elonmusk innovation AI
